<a href="https://colab.research.google.com/github/Cru1zzz3/python-parallel-programming-cookbook/blob/main/Python_Parallel_Programming_(Lab_6)_Udartsev_Stanislav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Проверяем версию компилятора NVIDIA CUDA

In [ ]:
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 4.2 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627577 sha256=ec22c1730cd95b1da08eb219c6dd479987fa85bd288091e572215c4f57efe8d1
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=fd43ca6700e7be573940ce4d29453a8435e581538bfb1286b13e28761af07b50
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


**Using the PyCUDA module**

Для того, чтобы инициализировать CUDA драйвер, необходимо сменить тип Runtime'a в Google Collab на тот, который поддерживает аппаратное ускорение GPU. 
Для этого необходимо: 

*   Нажать на вкладку `Runtime` и выбрать `Change runtime type`.
*   После этого в разделе `Hardware Acceleration`, выбрать `GPU` и нажать `Save`.




In [3]:
import pycuda.driver as drv 
drv.init() 
print("%d device(s) found." % drv.Device.count()) 
for ordinal in range(drv.Device.count()): 
  dev = drv.Device(ordinal) 
  print("Device #%d: %s" % (ordinal, dev.name())) 
  print("Compute Capability: %d.%d" % dev.compute_capability())     
  print("Total Memory: %s KB" % (dev.total_memory()//(1024)))

1 device(s) found.
Device #0: Tesla K80
Compute Capability: 3.7
Total Memory: 11715776 KB


После этого, должно инициализироваться видеоустройство Tesla K80

**How to build a PyCUDA application**

Вычисления могут производиться как на ЦПУ, так и на ГПУ. Как правило, с помощью на ЦПУ происходит подготовка данных, после чего данные передаются на ГПУ для дальнейших трудозатратных вычислений. Обработанные результирующие данные передаются обратно на ЦПУ, для их вывода. Одно из обязательных условий является выделения памяти на ГПУ заранее перед непосредственым выполнением вычислений. 

In [5]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

a = numpy.random.randn(5,5)
a = a.astype(numpy.float32)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
__global__ void doubleMatrix(float *a)
{
  int idx = threadIdx.x + threadIdx.y*4;
a[idx] *= 2; }
""")

func = mod.get_function("doubleMatrix")
func(a_gpu, block=(5,5,1))

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print("ORIGINAL MATRIX")
print(a)
print("DOUBLED MATRIX AFTER PyCUDA EXECUTION")
print(a_doubled)

ORIGINAL MATRIX
[[ 0.11767171  0.1937645  -1.0480621  -0.38260517 -0.45635247]
 [-0.30242795  0.6900911   0.76791215 -0.39162463 -0.5633676 ]
 [-0.91431695  0.48246655 -1.0379905   1.0718129  -1.7166617 ]
 [ 0.32136446 -1.1683393   0.7952338  -0.7712385   1.5996156 ]
 [ 0.75574934 -0.9920836   0.05320557  1.942601    2.012559  ]]
DOUBLED MATRIX AFTER PyCUDA EXECUTION
[[ 0.23534341  0.387529   -2.0961242  -0.76521033 -0.91270494]
 [-0.6048559   1.3801821   1.5358243  -0.78324926 -1.1267352 ]
 [-1.8286339   0.9649331  -2.075981    2.1436257  -3.4333234 ]
 [ 0.6427289  -2.3366785   1.5904676  -1.542477    3.1992311 ]
 [ 1.5114987  -0.9920836   0.05320557  1.942601    2.012559  ]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).



**Understanding the PyCUDA memory model
with matrix manipulation**

Не вся память одинакова по показателям скорости доступа в модели памяти ГПУ, но лучшей практикой является использование каждого
типа памяти наиболее эффективным образом. Основная идея состоит в том, чтобы свести к минимуму  доступ к глобальной памяти (global memory) за счет использования
общей памяти(shared memory). 

Этот метод обычно используется для разделения исходных данных
таким образом, чтобы мы позволяли блоку потоков выполнять свои вычисления в закрытом
подмножестве данных. Таким образом, потоки, присоединяющиеся к соответствующему блоку память, будут работать вместе, чтобы разгрузить глобальную память, и максимально утилизировать наиболее быстродейственную общую память.

In [9]:

import numpy as np
from pycuda import driver, compiler, gpuarray, tools
from pycuda.compiler import SourceModule

import pycuda.autoinit

kernel_code_template = """
__global__ void MatrixMulKernel(float *a, float *b, float *c)
{
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    float Pvalue = 0;
    for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
        float Aelement = a[ty * %(MATRIX_SIZE)s + k];
        float Belement = b[k * %(MATRIX_SIZE)s + tx];
        Pvalue += Aelement * Belement;
    }

    c[ty * %(MATRIX_SIZE)s + tx] = Pvalue;
}
"""

MATRIX_SIZE = 5

a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

c_cpu = np.dot(a_cpu, b_cpu)

a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)

c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

kernel_code = kernel_code_template % {
    'MATRIX_SIZE': MATRIX_SIZE 
}

mod = compiler.SourceModule(kernel_code)

matrixmul = mod.get_function("MatrixMulKernel")

matrixmul(
    a_gpu, b_gpu, 
    c_gpu, 
    block = (MATRIX_SIZE, MATRIX_SIZE, 1),
)

# print the results
print("-" * 80)
print("Matrix A (GPU):")
print(a_gpu.get())

print("-" * 80)
print("Matrix B (GPU):")
print(b_gpu.get())

print("-" * 80)
print("Matrix C (GPU):")
print(c_gpu.get())

print("-" * 80)
print("CPU-GPU difference:")
print(c_cpu - c_gpu.get())

np.allclose(c_cpu, c_gpu.get())


--------------------------------------------------------------------------------
Matrix A (GPU):
[[-1.6566936   0.56398535  1.9507375   0.58440423 -0.7316022 ]
 [ 1.7494041  -0.5206335   0.5521091  -1.5025228  -0.6369413 ]
 [ 0.28192496  0.77173406  0.56775284 -0.1784921  -0.35580882]
 [ 0.7989966   0.2856385  -0.8577809  -2.0350919  -0.24192981]
 [ 1.4297949   0.6130067  -0.17448708 -0.01680106  0.17822081]]
--------------------------------------------------------------------------------
Matrix B (GPU):
[[ 0.62587035  1.5087065   0.8595771  -0.6473098   0.65500885]
 [ 1.9399728   2.0890336  -0.5696015   1.1675851  -0.98925865]
 [ 1.0712496   1.119891    0.35546783 -0.33462828  1.8551363 ]
 [-0.28716367  1.6689091   0.45135853  1.4509321  -0.7902891 ]
 [ 0.5076282   0.9004284  -1.5363557  -0.74062985 -0.5335114 ]]
--------------------------------------------------------------------------------
Matrix C (GPU):
[[ 1.6077662   1.1798956   0.33589873  2.4679003   1.9042774 ]
 [ 0.7844725  

True

**Kernel invocations with GPUArray**

В библиотеке PyCUDA представлен класс pycuda.gpuarray.GPUArray который предоставляет высокоуровневый интервейс для выполнения вычислений с помощью CUDA (в отличии от pycuda.compiler.SourceModule, котороый вызывается компилятором nvcc из исходного кода СUDA)

In [13]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy

a_gpu = gpuarray.to_gpu(numpy.random.randn(4,4).astype(numpy.float32))
a_doubled = (2*a_gpu).get()
print("Doubled matrix using gpuarray call:\n", a_doubled)
print("Original matrix:\n",a_gpu)

Doubled matrix using gpuarray call:
 [[-0.76718646 -1.6005294  -1.2835188   1.3084855 ]
 [-0.7536936   1.927716   -4.4484     -0.49752158]
 [ 1.0747708  -1.7455909   1.684047    0.23103906]
 [ 1.2383196   3.658996   -2.9613283   1.3125074 ]]
Original matrix:
 [[-0.38359323 -0.8002647  -0.6417594   0.65424275]
 [-0.3768468   0.963858   -2.2242     -0.24876079]
 [ 0.5373854  -0.87279546  0.8420235   0.11551953]
 [ 0.6191598   1.829498   -1.4806641   0.6562537 ]]


**Evaluating element-wise expressions with
PyCUDA**

Функция PyCuda.elementwise.ElementwiseKernel похволяет нам выполнять сложные вычисления которые состоят из одного или более операндов за один вычислительный шаг.

В данном примере проводилось вычисления линейной комбинации двух случайно сгенерированных векторов (через pycuda.curandom) с помощью функции ElementwiseKernel. Входными параметрами данной функции стали: список аргументов, используемых для вычисления линейной комбинации 2х векторов, непосредственно определение вычисляемой линейной комбинации, а также наименование данной операции) 

In [14]:
import pycuda.autoinit
import numpy
from pycuda.curandom import rand as curand
from pycuda.elementwise import ElementwiseKernel
import numpy.linalg as la

input_vector_a = curand((50,))
input_vector_b = curand((50,))
mult_coefficient_a = 2
mult_coefficient_b = 5

linear_combination = ElementwiseKernel(
    "float a, float *x, float b, float *y, float *c",
    "c[i] = a*x[i] + b*y[i]",
    "linear_combination")

linear_combination_result = gpuarray.empty_like(input_vector_a)
linear_combination(mult_coefficient_a,
                   input_vector_a,\
                   mult_coefficient_b, input_vector_b,\
                   linear_combination_result)

print("INPUT VECTOR A =")
print(input_vector_a)

print("INPUT VECTOR B = ")
print(input_vector_b)

print("RESULTING VECTOR C = ")
print(linear_combination_result)

print("CHECKING THE RESULT EVALUATING THE DIFFERENCE VECTOR BETWEEN CAND THE LINEAR COMBINATION OF A AND B")
print("C - (%sA + %sB) = "%(mult_coefficient_a,mult_coefficient_b))

print(linear_combination_result -
      (mult_coefficient_a * input_vector_a + mult_coefficient_b * input_vector_b))

assert la.norm((linear_combination_result - \
                (mult_coefficient_a*input_vector_a +\
                 mult_coefficient_b*input_vector_b)).get()) < 1e-5

/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,
/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,


INPUT VECTOR A =
[0.16397922 0.2542899  0.1561198  0.7423383  0.48986718 0.24225038
 0.5527745  0.928087   0.9870533  0.42109317 0.32900068 0.32025725
 0.1596376  0.41115913 0.10089948 0.5432142  0.616877   0.17013712
 0.10183815 0.24852332 0.6760686  0.98465866 0.29470316 0.45642775
 0.24516201 0.5604631  0.6643754  0.07468423 0.7439757  0.315316
 0.40665123 0.52793115 0.7565336  0.20500064 0.2343749  0.8014761
 0.2337355  0.13296954 0.64089704 0.31119043 0.98951477 0.74160135
 0.8360364  0.56541276 0.6560929  0.8817107  0.01718229 0.5732893
 0.45194414 0.713942  ]
INPUT VECTOR B = 
[0.79965776 0.43139103 0.15423283 0.66269946 0.96225375 0.48722842
 0.05734671 0.46749467 0.3964474  0.276251   0.94695324 0.04347985
 0.99261177 0.60987836 0.28042185 0.8317375  0.19229913 0.6702712
 0.38508534 0.8897452  0.08817302 0.12089943 0.62499756 0.9237239
 0.187887   0.96184856 0.6624714  0.19917026 0.38990816 0.35291624
 0.21171579 0.36110634 0.4578505  0.6894811  0.63080394 0.80449146
 0.830141

/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,


**The MapReduce operation with PyCUDA**

PyCUDA предоставляет возможность выполнения операций методом MapReduce на ГПУ. Это возможно
с помощью встроенного в библиотеку метода pycuda.reduction.ReductionKernel

In [15]:
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
import numpy
from pycuda.reduction import ReductionKernel

vector_length = 400
input_vector_a = gpuarray.arange(vector_length, dtype=numpy.int)
input_vector_b = gpuarray.arange(vector_length, dtype=numpy.int)
dot_product = ReductionKernel(numpy.int,
                              arguments="int *x, int *y",
                              map_expr="x[i]*y[i]",
                              reduce_expr="a+b", neutral="0")

dot_product = dot_product (input_vector_a, input_vector_b).get()

print("INPUT VECTOR A")
print(input_vector_a)

print("INPUT VECTOR B")
print(input_vector_b)

print("RESULT DOT PRODUCT OF A * B")
print(dot_product)

/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,
/usr/local/lib/python3.7/dist-packages/pycuda/reduction.py:157: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  return SourceModule(src, options=options, keep=keep, no_extern_c=True)


INPUT VECTOR A
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242

/usr/local/lib/python3.7/dist-packages/pycuda/reduction.py:157: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  return SourceModule(src, options=options, keep=keep, no_extern_c=True)


**GPU programming with NumbaPro**

Numba - это компилятор Python, который предоставляет API на основе CUDA для написания программ CUDA.
Он предназначен для вычислительных задач, ориентированных на использование массивов данных, так же, как и широко используемая библиотека NumPy.
Параллелизм данных в задачах, ориентированных на массивы данных, естественным образом подходит для ускорителей, таких
как графические процессоры. NumbaPro понимает типы массивов NumPy и использует их для создания эффективного
скомпилированного кода для выполнения на графических процессорах или многоядерных процессорах.

В данном примере с помощью декоратора @guvectorize библиотеки numba мы определяем тип входных данных, а также каким образом манипулируем размерностями обрабатываемых матриц при умножении с помощью функции matmul()

In [5]:
from numba import guvectorize
import numpy as np

@guvectorize(['void(int64[:,:], int64[:,:], int64[:,:])'],
             '(m,n),(n,p)->(m,p)')
def matmul(A, B, C):
    m, n = A.shape
    n, p = B.shape
    for i in range(m):
        for j in range(p):
            C[i, j] = 0
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

dim = 10
A = np.random.randint(dim,size=(dim, dim))
B = np.random.randint(dim,size=(dim, dim))

C = matmul(A, B)
print("INPUT MATRIX A")
print(":\n%s" % A)
print("INPUT MATRIX B")
print(":\n%s" % B)
print("RESULT MATRIX C = A*B")
print(":\n%s" % C)

INPUT MATRIX A
:
[[7 3 9 6 2 6 9 4 6 8]
 [5 6 0 9 5 1 8 9 4 3]
 [5 3 8 4 3 1 7 7 1 9]
 [0 1 6 0 8 1 7 5 7 7]
 [5 5 0 3 3 7 4 3 1 8]
 [2 0 5 6 9 4 1 6 7 0]
 [6 1 0 8 8 8 0 3 2 2]
 [0 5 9 5 8 5 8 2 3 2]
 [0 1 4 4 6 7 2 9 4 1]
 [6 0 1 7 5 2 3 5 9 1]]
INPUT MATRIX B
:
[[6 6 6 5 9 8 3 7 8 4]
 [9 5 6 6 7 5 7 0 2 6]
 [4 4 2 1 4 6 1 4 1 5]
 [2 8 0 3 5 6 0 0 9 9]
 [5 7 3 4 3 3 3 2 1 4]
 [3 3 4 1 3 3 6 8 2 7]
 [7 3 3 7 8 2 4 1 5 6]
 [4 5 8 1 7 6 8 8 0 6]
 [5 2 5 9 9 1 7 5 0 0]
 [3 2 6 5 4 5 6 1 6 6]]
RESULT MATRIX C = A*B
:
[[278 248 245 255 360 273 251 216 232 321]
 [251 253 219 225 325 231 228 156 198 284]
 [224 209 213 186 280 241 204 158 184 271]
 [201 162 184 197 240 154 202 137  95 189]
 [186 166 186 163 219 180 196 138 162 228]
 [167 202 151 149 220 166 163 168  97 193]
 [153 208 144 131 196 179 147 158 158 220]
 [231 216 159 186 248 189 179 133 134 259]
 [157 177 164 117 201 159 185 179  78 207]
 [174 191 161 186 258 167 167 161 142 180]]


**Using GPU-accelerated libraries with
NumbaPro**

NumbaPro предоставляет оболочку Python для библиотек CUDA для численных вычислений. Каждый код,
использующий эти библиотеки, значительно ускорится без написания какого-либо дополнительного кода, специфичного для графического процессора.
Библиотека cuBLAS - предоставляет основные функции
линейной алгебры для работы на графическом процессоре, такие как операции над векторами, операции над матрицами и векторами, операциями над матрицами.

В данном примере представлена реализация алгоритма общего умножения матриц (GEMM), который представляет
собой процедуру для выполнения умножения матриц на графических процессорах NVIDIA:

In [7]:
import cupy
from cupy import cublas
import numpy as np
from timeit import default_timer as timer

dim = 10

def gemm():
  A = cupy.random.rand(dim, dim)
  B = cupy.random.rand(dim, dim)
  D = cupy.zeros_like(A, order='F')

  print("MATRIX A :")
  print(A)
  print("VECTOR B :")
  print(B)

  # NumPy
  start = timer()
  E = np.dot(A, B)
  numpy_time = timer() - start
  print("Numpy took %f seconds" % numpy_time)

  # cuBLAS
  start = timer()
  cublas.gemm('T', 'T', A, B, D, 1.0, 1.0)
  cuda_time = timer() - start

  print("RESULT MATRIX EVALUATED WITH CUBLAS")
  print(D)
  print("CUBLAS took %f seconds" % cuda_time)

  diff = np.abs(D - E)
  print("Maximum error %f" % np.max(diff))

def main():
  gemm()

if __name__ == '__main__':
  main()

MATRIX A :
[[0.10516194 0.55898318 0.62423573 0.21454811 0.88163658 0.68769649
  0.03899582 0.04067878 0.95279899 0.91470366]
 [0.59185705 0.12804578 0.89171253 0.35070279 0.01588767 0.75925402
  0.4226423  0.70868928 0.81258481 0.91337733]
 [0.70762026 0.24504798 0.20095707 0.53097925 0.77899371 0.10325536
  0.03842074 0.13368743 0.18296487 0.01186485]
 [0.32862373 0.53784013 0.94339876 0.4308364  0.17901376 0.00720555
  0.46994049 0.05660561 0.44849926 0.64717449]
 [0.2116461  0.78991917 0.81404575 0.322688   0.27266099 0.26849148
  0.56910238 0.96720697 0.00413226 0.5414472 ]
 [0.77945543 0.94755332 0.80392584 0.47441502 0.42985723 0.25004376
  0.71482539 0.05051171 0.87460246 0.70154276]
 [0.31990863 0.28220787 0.61881977 0.55184627 0.27329084 0.29894347
  0.08852946 0.83088912 0.46035398 0.04905428]
 [0.9393886  0.16564115 0.68263068 0.76415421 0.62502278 0.00125616
  0.71406214 0.41892215 0.57294203 0.27713827]
 [0.17708274 0.24297934 0.6190124  0.82863371 0.636678   0.36127385
 

**Using the PyOpenCL module**

Open Computing Language (OpenCL) - это фреймворк, используемый для разработки программ, работающих
на гетерогенных платформах, которые могут работать на ЦПУ или ГПУ,
созданных разными производителями.

In [8]:
!pip install pyopencl

     |████████████████████████████████| 879 kB 5.3 MB/s 


In [9]:
import pyopencl as cl


def print_device_info() :
  print('\n' + '=' * 60 + '\nOpenCL Platforms and Devices')
  for platform in cl.get_platforms():
    print('=' * 60)
    print('Platform - Name:  ' + platform.name)
    print('Platform - Vendor:  ' + platform.vendor)
    print('Platform - Version:  ' + platform.version)
    print('Platform - Profile:  ' + platform.profile)

    for device in platform.get_devices():
      print('    ' + '-' * 56)
      print('    Device - Name:  ' \
            + device.name)
      print('    Device - Type:  ' \
            + cl.device_type.to_string(device.type))
      print('    Device - Max Clock Speed:  {0} Mhz'\
            .format(device.max_clock_frequency))
      print('    Device - Compute Units:  {0}'\
            .format(device.max_compute_units))
      print('    Device - Local Memory:  {0:.0f} KB'\
            .format(device.local_mem_size/1024.0))
      print('    Device - Constant Memory:  {0:.0f} KB'\
            .format(device.max_constant_buffer_size/1024.0))
      print('    Device - Global Memory: {0:.0f} GB'\
            .format(device.global_mem_size/1073741824.0))
      print('    Device - Max Buffer/Image Size: {0:.0f} MB'\
            .format(device.max_mem_alloc_size/1048576.0))
      print('    Device - Max Work Group Size: {0:.0f}'\
            .format(device.max_work_group_size))
  print('\n')

if __name__ == "__main__":
  print_device_info()
	


OpenCL Platforms and Devices
Platform - Name:  NVIDIA CUDA
Platform - Vendor:  NVIDIA Corporation
Platform - Version:  OpenCL 1.2 CUDA 11.2.109
Platform - Profile:  FULL_PROFILE
    --------------------------------------------------------
    Device - Name:  Tesla K80
    Device - Type:  ALL | GPU
    Device - Max Clock Speed:  823 Mhz
    Device - Compute Units:  13
    Device - Local Memory:  48 KB
    Device - Constant Memory:  64 KB
    Device - Global Memory: 11 GB
    Device - Max Buffer/Image Size: 2860 MB
    Device - Max Work Group Size: 1024




**How to build a PyOpenCL application**

Первым шагом для создания программы на PyOpenCL является
кодирование приложения на основном хосте. Фактически, это выполняется на главном компьютере (как правило, на
компьютере пользователя), а затем он отправляет приложение на подключенные устройства ГПУ.

In [10]:
import numpy as np
import pyopencl as cl
import numpy.linalg as la

vector_dimension = 100

vector_a = np.random.randint(vector_dimension, size=vector_dimension, dtype=np.uint8)
vector_b = np.random.randint(vector_dimension, size=vector_dimension, dtype=np.uint8)

platform = cl.get_platforms()[0]
device = platform.get_devices()[0]

context = cl.Context([device])
queue = cl.CommandQueue(context)

mf = cl.mem_flags
a_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR,
hostbuf=vector_a)
b_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR,
hostbuf=vector_b)

program = cl.Program(context, """
__kernel void vectorSum(__global const int *a_g, __global const int
*b_g, __global int *res_g) {
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

res_g = cl.Buffer(context, mf.WRITE_ONLY, vector_a.nbytes)
program.vectorSum(queue, vector_a.shape, None, a_g, b_g, res_g)

res_np = np.empty_like(vector_a, dtype=np.uint8)
cl.enqueue_copy(queue, res_np, res_g)

print("PyOPENCL SUM OF TWO VECTORS")
print("Platform Selected = %s" %platform.name )
print("Device Selected = %s" %device.name)
print("VECTOR LENGTH = %s" %vector_dimension)
print("INPUT VECTOR A")
print(vector_a)
print("INPUT VECTOR B")
print(vector_b)
print("OUTPUT VECTOR RESULT A + B ")
print(res_np)

assert(la.norm(res_np - (vector_a + vector_b))) < 1e-5

PyOPENCL SUM OF TWO VECTORS
Platform Selected = NVIDIA CUDA
Device Selected = Tesla K80
VECTOR LENGTH = 100
INPUT VECTOR A
[23 65 98 78 41 29 79 90 20 16 98 74 12  9 58  0 94 90 37 98 78 43 40 46
 19 30 95 32 52 91 22 21 65 14 70 14 24 13 93 66 16 34 80 99 60 18 44 44
 36 92 55 57 29  6 74 86 84 55 10 44 75 43 29 65 73 99  7 80 55 92 64 70
 16 80 85  3 56 52 13 40 23 45 38 94 18 41 63 71 65 36 23 15 86 36 31 40
 97 50 75 78]
INPUT VECTOR B
[99 29 18 70 36 49 70 13 28  7  6 43 57 78 14 77 33 91 85 69 20 13 60 82
 82 86 75 93 14 13 81 13 72 26 49 81 99 99 34 35 44 25  1 62 11 39 51 66
 51 92 69 62 32 66 23 78  8 16 53 60 71 62  8 46 62 22 91  4 11  1 17  9
 25 15 14 65 87 72 97 22 62 51 24  7 38 32 71 40 11 17 49 55 52 51 53 73
 74 67 15 30]
OUTPUT VECTOR RESULT A + B 
[122  94 116 148  77  78 149 103  48  23 104 117  69  87  72  77 127 181
 122 167  98  56 100 128 101 116 170 125  66 104 103  34 137  40 119  95
 123 112 127 101  60  59  81 161  71  57  95 110  87 184 124 119  61  72
  9

**Evaluating element-wise expressions with
PyOpenCL**

Подобно PyCUDA, PyOpenCL предоставляет функциональные возможности
класса pyopencl.elementwise, которые позволяют нам проводить вычисления над сложными выражениями за один вычислительный проход.

In [11]:
import pyopencl as cl
import pyopencl.array as cl_array
import numpy as np

context = cl.create_some_context()
queue = cl.CommandQueue(context)

vector_dimension = 100
vector_a = cl_array.to_device(queue,  np.random.randint(vector_dimension, size=vector_dimension, dtype=np.uint8))
vector_b = cl_array.to_device(queue,  np.random.randint(vector_dimension, size=vector_dimension, dtype=np.uint8))
result_vector = cl_array.empty_like(vector_a)

elementWiseSum = cl.elementwise.ElementwiseKernel(context, "int *a, int *b, int *c", "c[i] = a[i] + b[i]", "sum")
elementWiseSum(vector_a, vector_b, result_vector)

print("PyOpenCL ELEMENTWISE SUM OF TWO VECTORS")
print("VECTOR LENGTH = %s" %vector_dimension)
print("INPUT VECTOR A")
print(vector_a)
print("INPUT VECTOR B")
print(vector_b)
print("OUTPUT VECTOR RESULT A + B ")
print(result_vector)

PyOpenCL ELEMENTWISE SUM OF TWO VECTORS
VECTOR LENGTH = 100
INPUT VECTOR A
[30 72 81 34 11 75 63 12 37 82 20 42 66 64  9 29 34 20 63 57  9 40 48 33
 75 42 20 49 55 90 95 92 54 99 28 21 61 23 14 27 68  8 41  3 33 18 90 44
  2 98 79 96 18 67 64 47 34 16 99 78 97 98 33 48  8 74 18  3  7 28 49 82
 43 38 24 56 43  4 57 79 26  8 60 23 12 41 11 67  7 90 17 26 38 79 69 68
 29 70 31 15]
INPUT VECTOR B
[90 40 70 26 58 25 72 52 67  6 97 95  8 53 29  3 12 79 80 88  8 43 52 16
 56  9  0 48 48 63 45 95 18 17 52 76 83 13 92  2 48 71 91  4 15 80 75 41
 66 68 10 42 28  0 51 16 37 98  2 92 10 76 69 82 12 33  7 31 42 97 74 84
 75 35 32 67 51 95 64 85 77 33 92 69 59 12 64 49 92 40 77 84 69 35 78 19
 18 81  4 44]
OUTPUT VECTOR RESULT A + B 
[120 112 151  60  69 100 135  64 104  88 117 137  74 117  38  32  46  99
 143 145  17  83 100  49 131  51  20  97 103 153 140 187  72 116  80  97
 144  36 106  29 116  79 132   7  48  98 165  85  68 166  89 138  46  67
 115  63  71 114 101 170 107 174 102 130  20 107  2

**Testing your GPU application with PyOpenCL**

В данном примере выполняется оценка и сравнения времени вычисления суммы двух
векторов с элементами, представленными в виде чисел с плавающей точкой. Для
сравнения, одна и та же операция была реализована в двух отдельных функциях.
Первая функция использует только ЦПУ, в то время как вторая написана с использованием PyOpenCL и использует ГПУ для вычислений.

Как можно видеть из результата, скорость вычисление на ГПУ значительно превосходят вычисления на ЦПУ для решения данной задачи.

In [12]:
from time import time
import pyopencl as cl
import numpy as np
import numpy.linalg as la

a = np.random.rand(10000).astype(np.float32)
b = np.random.rand(10000).astype(np.float32)

def test_cpu_vector_sum(a, b):
  c_cpu = np.empty_like(a)
  cpu_start_time = time()
  for i in range(10000):
    for j in range(10000):
      c_cpu[i] = a[i] + b[i]
  cpu_end_time = time()
  print("CPU Time: {0} s".format(cpu_end_time - cpu_start_time))
  return c_cpu

def test_gpu_vector_sum(a, b):
  platform = cl.get_platforms()[0]
  device = platform.get_devices()[0]
  context = cl.Context([device])
  queue = cl.CommandQueue(context, properties=cl.command_queue_properties.PROFILING_ENABLE)

  a_buffer = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=a)
  b_buffer = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=b)
  c_buffer = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, b.nbytes)

  program = cl.Program(context, """
  __kernel void sum(__global const float *a,
                    __global const float *b,
                    __global float *c)
  {
      int i = get_global_id(0);
      int j;
      for(j = 0; j < 10000; j++)
      {
          c[i] = a[i] + b[i];
      }
  }""").build()

  gpu_start_time = time()
  event = program.sum(queue, a.shape, None, a_buffer, b_buffer, c_buffer)
  event.wait()
  elapsed = 1e-9*(event.profile.end - event.profile.start)
  print("GPU Kernel evaluation Time: {0} s".format(elapsed))
  c_gpu = np.empty_like(a)
  cl._enqueue_read_buffer(queue, c_buffer, c_gpu).wait()
  gpu_end_time = time()
  print("GPU Time: {0} s".format(gpu_end_time - gpu_start_time))

  return c_gpu

if __name__ == "__main__":
  cpu_result = test_cpu_vector_sum(a, b)
  gpu_result = test_gpu_vector_sum(a, b)
  assert (la.norm(cpu_result - gpu_result)) < 1e-5

CPU Time: 47.169475078582764 s
GPU Kernel evaluation Time: 0.0068231360000000005 s
GPU Time: 0.009328126907348633 s


**Выводы:**

В ходе лабораторной работы было рассмотрено выполнение распределенных вычислений, а также архитектура модели памяти на графических процессорах поддерживающих технологию CUDA от NVIDA. Продемонстрировано взаимодействие с графическими процессорами с помощью библиотек PyCUDA, Numba, PyOpenCL. Было проведено тестирование и оценка использования вычислений на графических процессорах над различными математическими операциями.